# Trânsito de Vênus de 5 a 6 de junho de 2012
_Imagens da sonda SDO/AIA (Atmospheric Imaging Assembly) no comprimento de onda de 4500 Å (visível) ou 1600 Å/1700 Å (UV), que mostram bem o disco solar e o planeta escuro passando na frente._

Dependências: sunpy, matplotlib, astropy, numpy.

O canal de 1700 Å é uma boa escolha para esse evento; 4500 Å está disponível em algumas bases, mas menos comum.

O método np.nansum ignora valores NaN que podem estar na borda.

Se quiser usar toda a imagem, substitua submap por m.data.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sunpy.net import Fido, attrs as a
from sunpy.map import Map
from sunpy.time import parse_time
from astropy.time import Time
import astropy.units as u

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Buscar dados do SDO/AIA durante o trânsito de Vênus (5-6 de junho de 2012)
start_time = "2012-06-05 22:00"
end_time = "2012-06-06 04:00"
wavelength = 1700 * u.angstrom  # Visível, pode trocar por 4500 se quiser

result = Fido.search(
    a.Time(start_time, end_time),
    a.Instrument.aia,
    a.Wavelength(wavelength),
    a.Sample(10 * u.minute) # amostragem a cada 10 minutos
)

print(f"Imagens encontradas: {len(result[0])}")

# 2. Download (pequeno conjunto para teste)
downloaded = Fido.fetch(result[0][:30])  # baixa apenas as 30 primeiras para evitar excesso

# 3. Processar cada imagem e calcular o brilho total (soma dos pixels)
times = []
fluxes = []

for filepath in downloaded:
    m = Map(filepath)
    
    # Opcional: recortar o disco solar para excluir bordas ou artefatos
    submap = m.submap(
        bottom_left=m.center - [500*u.arcsec, 500*u.arcsec],
        top_right=m.center + [500*u.arcsec, 500*u.arcsec]
    )
    
    # Brilho total = soma de intensidades
    total_flux = np.nansum(submap.data)
    fluxes.append(total_flux)
    times.append(parse_time(m.date))

# 4. Plotar curva de luz
plt.figure(figsize=(10, 5))
plt.plot_date(Time(times).plot_date, fluxes, '-o', label='Fluxo Solar')
plt.title("Curva de Luz do Trânsito de Vênus (SDO/AIA 1700 Å)")
plt.xlabel("Tempo (UTC)")
plt.ylabel("Fluxo Integrado (adu)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


Imagens encontradas: 36


Files Downloaded:   0%|          | 0/30 [00:00<?, ?file/s]